## Corpus

Polarity Dataset. Pang/Lee ACL 2004

http://www.cs.cornell.edu/people/pabo/movie-review-data/

http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz

Based on : https://github.com/cezannec/CNN_Text_Classification/blob/master/CNN_Text_Classification.ipynb

## Load Corpus

In [1]:
from corpus import prepare_corpus

corpus = prepare_corpus()

prepare_corpus: 100%|██████████| 1000/1000 [00:04<00:00, 208.77it/s]


In [2]:
len(corpus)

2000

## Embedding - Word2Vec

Can either try pretraiend embeddings from google news dataset or train a new embedding from the existing corpus. Let's try it on the existing corpus.

In [3]:
import multiprocessing
from gensim.models import Word2Vec

sentences = [s[1] for s in corpus]

# dims for the embedding
# embedding dim will define the number of outputs per convolution!
EMBEDDING_DIMS = 300


cores = multiprocessing.cpu_count()

# keep one cpu core free or some operating systems may kill the process :P
# ignore words which have a frequency < 5
corpus_embedding = Word2Vec(
    vector_size=EMBEDDING_DIMS, 
    workers=cores-1, max_vocab_size=100000,
    min_count=5)

# build vocab
corpus_embedding.build_vocab(sentences)

# train for 50 epochs, can always change later!
%time corpus_embedding.train(sentences, total_examples=len(sentences), epochs=50)

Wall time: 14.7 s


(32383069, 37305350)

### Vocabulary

In [4]:
# list of all the words word2vec has processed
vocabulary = corpus_embedding.wv.index_to_key
vocab_len = len(vocabulary)

In [5]:
vocab_len

14718

In [6]:
vocabulary[:10]

["'s", '``', 'film', "n't", 'movie', 'one', 'like', 'even', 'good', 'time']

## Encode all tokens with indices from embedding

In [7]:
from tqdm import tqdm

def encode_corpus_tokens_with_embed_idx(corpus):
    encoded_corpus = list()
    for i in tqdm(range(len(corpus)), desc="encode_tokens_with_embed_idx"):
        idxs = []
        label, tokens = corpus[i]

        for token in tokens:
            try:
                idx = corpus_embedding.wv.key_to_index[token]
            except:
                # if token isn't in the vocab
                idx = 0

            idxs.append(idx)
        
        
        encoded_corpus.append((label, idxs))



    return encoded_corpus

In [8]:
encoded_corpus = encode_corpus_tokens_with_embed_idx(corpus)

encode_tokens_with_embed_idx: 100%|██████████| 2000/2000 [00:00<00:00, 11759.79it/s]


## Padding

Left pad with 0

However we need a sequence length. 

In [9]:
# get max sequences length
max_seq_len = max(len(s) for s in sentences)
max_seq_len

1477

In [10]:
import numpy as np

def pad_tokens(encoded_corpus, seq_len=max_seq_len):
    padded = np.zeros(
        (len(encoded_corpus), seq_len),
        dtype=np.int32
    )

    for i in tqdm(range(len(corpus)), desc="pad"):
        tokens = encoded_corpus[i][1]

        # nltk's stopwords are a bit agrressive, ignore token lists with 0 size
        if len(tokens) == 0:
            continue

        padded[i, -len(tokens):] = np.array(tokens)

    return padded

In [11]:
padded_tokens = pad_tokens(encoded_corpus)

pad: 100%|██████████| 2000/2000 [00:00<00:00, 35715.66it/s]


## Input and Labels?

In [12]:
X = padded_tokens # input
y = np.array([c[0] for c in encoded_corpus])  #label

In [13]:
print(X.shape)
print(y.shape)

(2000, 1477)
(2000,)


## Split data

In [14]:

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    X, y, random_state=42, train_size=0.8
)

In [15]:
# https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, random_state=42)

In [16]:
print(f"x_train = {x_train.shape} # y_train = {y_train.shape}")
print(f"x_val = {x_val.shape} # y_val = {y_val.shape}")
print(f"x_test = {x_test.shape} # y_test = {y_test.shape}")

x_train = (1280, 1477) # y_train = (1280,)
x_val = (320, 1477) # y_val = (320,)
x_test = (400, 1477) # y_test = (400,)


## Convert to TensorData

In [17]:
import torch
from torch.utils.data import TensorDataset

training_data = TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
val_data = TensorDataset(torch.from_numpy(x_val), torch.from_numpy(y_val))

## DataLoader for Torch

Let torch handle the shuffling and etc yada stuff for MiniBatch

Why MiniBatch? Dataset is big and feeding everything at once won't generalize well. (Even if the machine can handle it!)

In [18]:
from torch.utils.data import DataLoader

# define a batch size
batch_size = 64

train_loader = DataLoader(training_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)

## CNN Model

This model is based on https://arxiv.org/abs/1408.5882

In [19]:
import torch.nn as nn
import torch.nn.functional as F

class SentimentClassifierCNN(nn.Module):
    def __init__(self, freeze_embeddings=True):
        super(SentimentClassifierCNN, self).__init__()
        
        # properties
        self.kernel_sizes = [3,4,5]
        self.num_filters = 100
        self.embedding_dim = EMBEDDING_DIMS
        self.output_size = 1
        self.vocab_size=vocab_len

        # convert embeddings to tensors!
        self.corpus_embedding = torch.from_numpy(corpus_embedding.wv.vectors)

        # neural network 

        # embedding layer
        # by default we're freezing embeddings
        self.embedding = nn.Embedding.from_pretrained(self.corpus_embedding, freeze=freeze_embeddings)

        # conv layers
        # 3 conv layers, since 3 kernel sizes
        self.conv1d = nn.ModuleList([
            nn.Conv2d(1, self.num_filters, (k, self.embedding_dim), padding=(k - 2, 0))

            for k in self.kernel_sizes
        ])

        # final linear layer
        self.linear = nn.Linear(len(self.kernel_sizes) * self.num_filters, self.output_size)

        # dropout and sigmoid
        # why sigmoid? Well, binary classification task!
        self.dropout = nn.Dropout(0.1)
        self.sigmoid = nn.Sigmoid()

    # helper 
    def conv_and_pool(self, x, conv):
        """
        Convolutional + max pooling layer
        """
        # squeeze last dim to get size: (batch_size, num_filters, conv_seq_length)
        x = F.relu(conv(x)).squeeze(3)
        
        # 1D pool over conv_seq_length
        # squeeze to get size: (batch_size, num_filters)
        x_max = F.max_pool1d(x, x.size(2)).squeeze(2)
        return x_max

    def forward(self, x):
        embeds = self.embedding(x)
        embeds = embeds.unsqueeze(1) # reshape for conv (vector to matrix)

        conv_out =  [self.conv_and_pool(embeds, conv) for conv in self.conv1d]

        # concate convolution outputs as a "vector"
        out = torch.cat(conv_out, 1)
        # apply dropout
        out = self.dropout(out)

        # linear 
        out = self.linear(out)

        return self.sigmoid(out)

In [20]:
cnn = SentimentClassifierCNN()
print(cnn)

SentimentClassifierCNN(
  (embedding): Embedding(14718, 300)
  (conv1d): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1), padding=(1, 0))
    (1): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1), padding=(2, 0))
    (2): Conv2d(1, 100, kernel_size=(5, 300), stride=(1, 1), padding=(3, 0))
  )
  (linear): Linear(in_features=300, out_features=1, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (sigmoid): Sigmoid()
)


## Hyperparams

In [21]:
learning_rate = 0.001

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

## Create Device
Using accelerate from huggingface
https://huggingface.co/docs/accelerate/index.html


In [22]:
from accelerate import Accelerator

accelerator = Accelerator()
device = accelerator.device

print(device)

cuda


## Add model, dataloader, optimizer and dataset to device

In [23]:
cnn, train_loader, val_loader, optimizer = accelerator.prepare(
    cnn, train_loader, val_loader, optimizer
)

## Train

In [24]:
epochs = 2

def train_cnn(model, train_loader, val_loader, epochs, optimizer, loss_fn, accl=accelerator):
    print_counter = 0 # print loss for each 10th count

    for e in tqdm(range(epochs), desc=f"train_cnn_for_{epochs}_epochs"):
        model.train()
        for input, label in train_loader:
            print_counter += 1
            # zero gradients
            model.zero_grad()

            # forward pass
            output = model(input)

            # backprop
            loss = loss_fn(output.squeeze(), label.float())
            accl.backward(loss)
            optimizer.step()

            # log loss 
            if print_counter % 10 == 0:
                validation_losses = []
                
                model.eval() # switch mode
                with torch.no_grad():
                    for val_input, val_label in val_loader:
                        val_output = model(val_input)
                        val_loss = loss_fn(val_output.squeeze(), val_label.float())
                        validation_losses.append(val_loss.item())
                    print(f"\nEpoch: {e + 1}/{epochs}\tStep: {print_counter}\tTrain Loss: {loss.item()}\tValidation Loss: {np.mean(validation_losses)}")

                model.train()

            


%time train_cnn(model=cnn, train_loader=train_loader, val_loader=val_loader, epochs=epochs, optimizer=optimizer, loss_fn=loss_fn)

train_cnn_for_2_epochs:   0%|          | 0/2 [00:00<?, ?it/s]
Epoch: 1/2	Step: 10	Train Loss: 0.6042454242706299	Validation Loss: 0.5925650477409363
train_cnn_for_2_epochs:  50%|█████     | 1/2 [00:04<00:04,  4.47s/it]
Epoch: 1/2	Step: 20	Train Loss: 0.6561560034751892	Validation Loss: 0.520997965335846

Epoch: 2/2	Step: 30	Train Loss: 0.29841870069503784	Validation Loss: 0.48724937438964844
train_cnn_for_2_epochs: 100%|██████████| 2/2 [00:07<00:00,  3.67s/it]
Epoch: 2/2	Step: 40	Train Loss: 0.3817400336265564	Validation Loss: 0.47735190987586973
Wall time: 7.34 s



## Inference

In [25]:
test_x_tensor = torch.from_numpy(x_test)
test_x_tensor = test_x_tensor.to(device)

def classify_sentiment(model, test_data):
    model.eval()
    with torch.no_grad():

        out = model(test_data)
        out = torch.round(out.squeeze())
    	
        return out.cpu().detach().numpy()

In [26]:
y_pred = classify_sentiment(cnn, test_x_tensor)

## Evaluation

In [27]:
from sklearn.metrics import classification_report

print(classification_report(y_pred=y_pred, y_true=y_test))

              precision    recall  f1-score   support

           0       0.71      0.77      0.74       199
           1       0.75      0.69      0.72       201

    accuracy                           0.73       400
   macro avg       0.73      0.73      0.73       400
weighted avg       0.73      0.73      0.73       400

